<a href="https://colab.research.google.com/github/dh610/Advanced-AI/blob/main/AirNet_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pip install fasttext
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-40s7z62c
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-40s7z62c
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=f166ff26b18337b019dcf6952f1956b4ba8b725688887465a4f7dafad31d988b
  Stored in directory: /tmp/pip-ephem-wheel-cache-aik6ja3m/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
  Attempting uninstall: clip
    Found existing installation: clip 0.2.0
    Uninstalling clip-0.2.0:
      Successfully uninstalled clip-0.2.0


In [28]:
import sys
import os

os.chdir("/content/drive/MyDrive/Advanced-AI/")
!git pull
os.chdir("/content/drive/MyDrive/Advanced-AI/Ajou_Restore")

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 324 bytes | 1024 bytes/s, done.
From https://github.com/dh610/Advanced-AI
   7e251cd..a3937a4  main       -> origin/main
Updating 7e251cd..a3937a4
Fast-forward
 Ajou_Restore/option.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [31]:
!python3 train_text.py --epochs_encoder=0

Start training...
  7% 4/57 [00:14<03:07,  3.53s/it]
Traceback (most recent call last):
  File "/content/drive/MyDrive/Advanced-AI/Ajou_Restore/train_text.py", line 57, in <module>
    loss.backward()
  File "/usr/local/lib/python3.10/dist-packages/torch/_tensor.py", line 521, in backward
    torch.autograd.backward(
  File "/usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py", line 289, in backward
    _engine_run_backward(
  File "/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py", line 769, in _engine_run_backward
    return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt


In [ ]:
!pip install scikit-video

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 80.1 MB/s eta 0:00:00


In [ ]:
!python test.py --mode=1 --output_path="$output_path" --ckpt_path="$ckpt_path" --derain_path="$derain_path"

/content/drive/MyDrive/Advanced-AI/AirNet/test.py:101: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(ckpt_path, map_location=torch.device(opt.